# Assignment No 3
### Name: Jawad Ahmed
### Roll No: 20P-0165
### Section: BCS-6A

In [5]:
import re

In [6]:
def read_file(file_name):
    pattern = re.compile(r'(\d+),(\{[^}]*\}),(\d+\.\d+)')

    result = {}

    with open('data0.txt', 'r') as f:
        data = f.read()

    matches = pattern.findall(data)

    for match in matches:
        key = int(match[0])
        values = match[1]
        if values:
            values = [{int(val)} for val in values.strip('{}').split(',') if val]
        else:
            values = [set()]
        result.setdefault(key, []).append((values, float(match[2])))
    return result

In [7]:
result = read_file('data0.txt')

In [8]:
print(result)

{1: [([], 153.466), ([{3}], 96.093), ([{4}], 97.913), ([{5}], 99.835)], 2: [([], 141.023), ([{3}], 122.446), ([{4}], 121.576), ([{5}], 123.398)], 3: [([], 169.482), ([{1}], 112.109), ([{2}], 150.906), ([{1}, {2}], 107.516), ([{4}], 51.681), ([{5}], 41.775)], 4: [([], 169.482), ([{1}], 113.929), ([{2}], 150.036), ([{1}, {2}], 108.982), ([{3}], 51.681), ([{5}], 36.188)], 5: [([], 169.802), ([{1}], 116.171), ([{2}], 152.178), ([{1}, {2}], 111.473), ([{3}], 42.096), ([{4}], 36.508)]}


In [13]:
def print_dict(a):
    for vertex, value in a.items():
        print(f"Vertex {vertex}:")
        for i, (parents, cost) in enumerate(value):
            parents_str = "{" + ", ".join(str(p) for p in parents) + "}" if parents else "∅"
            print(f"  {i+1}. Parents: {parents_str}, Cost: {cost:.3f}")


In [14]:
print_dict(result)

Vertex 1:
  1. Parents: ∅, Cost: 153.466
  2. Parents: {{3}}, Cost: 96.093
  3. Parents: {{4}}, Cost: 97.913
  4. Parents: {{5}}, Cost: 99.835
Vertex 2:
  1. Parents: ∅, Cost: 141.023
  2. Parents: {{3}}, Cost: 122.446
  3. Parents: {{4}}, Cost: 121.576
  4. Parents: {{5}}, Cost: 123.398
Vertex 3:
  1. Parents: ∅, Cost: 169.482
  2. Parents: {{1}}, Cost: 112.109
  3. Parents: {{2}}, Cost: 150.906
  4. Parents: {{1}, {2}}, Cost: 107.516
  5. Parents: {{4}}, Cost: 51.681
  6. Parents: {{5}}, Cost: 41.775
Vertex 4:
  1. Parents: ∅, Cost: 169.482
  2. Parents: {{1}}, Cost: 113.929
  3. Parents: {{2}}, Cost: 150.036
  4. Parents: {{1}, {2}}, Cost: 108.982
  5. Parents: {{3}}, Cost: 51.681
  6. Parents: {{5}}, Cost: 36.188
Vertex 5:
  1. Parents: ∅, Cost: 169.802
  2. Parents: {{1}}, Cost: 116.171
  3. Parents: {{2}}, Cost: 152.178
  4. Parents: {{1}, {2}}, Cost: 111.473
  5. Parents: {{3}}, Cost: 42.096
  6. Parents: {{4}}, Cost: 36.508


# Hill Climbing 

In [59]:
import math

# Define the data
data = [(1, [([], 153.466), ([3], 96.093), ([4], 97.913), ([5], 99.835)]),
        (2, [([], 141.023), ([3], 122.446), ([4], 121.576), ([5], 123.398)]),
        (3, [([], 169.482), ([1], 112.109), ([2], 150.906), ([1, 2], 107.516), ([4], 51.681), ([5], 41.775)]),
        (4, [([], 169.482), ([1], 113.929), ([2], 150.036), ([1, 2], 108.982), ([3], 51.681), ([5], 36.188)]),
        (5, [([], 169.802), ([1], 116.171), ([2], 152.178), ([1, 2], 111.473), ([3], 42.096), ([4], 36.508)])]

def calculate_cost(ordering):
    total_cost = 0
    for vertex, neighbors in data:
        parent_set = set()
        for neighbor in neighbors:
            if set(tuple(neighbor[0])).issubset(set(ordering[:ordering.index(vertex)])):
                parent_set.add(tuple(neighbor[0]))
        if not parent_set:
            total_cost += neighbors[0][1] # Use the default cost if there are no parents
        else:
            min_cost = float("inf")
            for parents in parent_set:
                for neighbor in neighbors:
                    if neighbor[0] == list(parents):
                        cost = neighbor[1]
                        if cost < min_cost:
                            min_cost = cost
            total_cost += min_cost
    return total_cost

def get_neighbors(ordering):
    neighbors = []
    for i in range(len(ordering)):
        for j in range(i+1, len(ordering)):
            neighbor = ordering[:i] + [ordering[j]] + ordering[i+1:j] + [ordering[i]] + ordering[j+1:]
            neighbors.append(neighbor)
    return neighbors

def hill_climbing(ordering):
    current_ordering = ordering
    current_cost = calculate_cost(current_ordering)
    while True:
        neighbors = get_neighbors(current_ordering)
        found_better_neighbor = False
        for neighbor in neighbors:
            neighbor_cost = calculate_cost(neighbor)
            if neighbor_cost < current_cost:
                current_ordering = neighbor
                current_cost = neighbor_cost
                found_better_neighbor = True
        if not found_better_neighbor:
            break
    return current_ordering, current_cost

ordering = [5, 3, 1, 4, 2]
best_ordering, best_cost = hill_climbing(ordering)

print("Best Ordering:", best_ordering)
print("Best Cost:", best_cost)


Best Ordering: [5, 3, 1, 4, 2]
Best Cost: 465.43399999999997


# Simulated Annealing

In [28]:
import math
import random

# Define the data
data = [(1, [([], 153.466), ([3], 96.093), ([4], 97.913), ([5], 99.835)]),
        (2, [([], 141.023), ([3], 122.446), ([4], 121.576), ([5], 123.398)]),
        (3, [([], 169.482), ([1], 112.109), ([2], 150.906), ([1, 2], 107.516), ([4], 51.681), ([5], 41.775)]),
        (4, [([], 169.482), ([1], 113.929), ([2], 150.036), ([1, 2], 108.982), ([3], 51.681), ([5], 36.188)]),
        (5, [([], 169.802), ([1], 116.171), ([2], 152.178), ([1, 2], 111.473), ([3], 42.096), ([4], 36.508)])]

def calculate_cost(ordering):
    total_cost = 0
    for vertex, neighbors in data:
        parent_set = set()
        for neighbor in neighbors:
            if set(tuple(neighbor[0])).issubset(set(ordering[:ordering.index(vertex)])):
                parent_set.add(tuple(neighbor[0]))
        if not parent_set:
            total_cost += neighbors[0][1] # Use the default cost if there are no parents
        else:
            min_cost = float("inf")
            for parents in parent_set:
                for neighbor in neighbors:
                    if neighbor[0] == list(parents):
                        cost = neighbor[1]
                        if cost < min_cost:
                            min_cost = cost
            total_cost += min_cost
    return total_cost

def simulated_annealing(ordering, T=1.0, cooling_rate=0.0001, stopping_T=0.0001):
    best_ordering = ordering
    best_cost = calculate_cost(best_ordering)
    current_ordering = ordering
    current_cost = calculate_cost(current_ordering)
    while T > stopping_T:
        i = random.randint(0, len(ordering) - 1)
        j = random.randint(0, len(ordering) - 1)
        new_ordering = current_ordering[:i] + current_ordering[i:j+1][::-1] + current_ordering[j+1:]
        new_cost = calculate_cost(new_ordering)
        delta_E = new_cost - current_cost
        if delta_E < 0 or random.random() < math.exp(-delta_E / T):
            current_ordering = new_ordering
            current_cost = new_cost
        if current_cost < best_cost:
            best_ordering = current_ordering
            best_cost = current_cost
        T *= 1 - cooling_rate
    return best_ordering, best_cost

ordering = [5, 3, 1, 4, 2]
best_ordering, best_cost = simulated_annealing(ordering)

print("Best Ordering:", best_ordering)
print("Best Cost:", best_cost)


Best Ordering: [5, 3, 1, 4, 2]
Best Cost: 465.43399999999997


# Breadth First Search

In [54]:
import math
from collections import deque

# Define the data
data = [(1, [([], 153.466), ([3], 96.093), ([4], 97.913), ([5], 99.835)]),
        (2, [([], 141.023), ([3], 122.446), ([4], 121.576), ([5], 123.398)]),
        (3, [([], 169.482), ([1], 112.109), ([2], 150.906), ([1, 2], 107.516), ([4], 51.681), ([5], 41.775)]),
        (4, [([], 169.482), ([1], 113.929), ([2], 150.036), ([1, 2], 108.982), ([3], 51.681), ([5], 36.188)]),
        (5, [([], 169.802), ([1], 116.171), ([2], 152.178), ([1, 2], 111.473), ([3], 42.096), ([4], 36.508)])]

def calculate_cost(ordering):
    total_cost = 0
    for vertex, neighbors in data:
        parent_set = set()
        for neighbor in neighbors:
            if set(tuple(neighbor[0])).issubset(set(ordering[:ordering.index(vertex)])):
                parent_set.add(tuple(neighbor[0]))
        if not parent_set:
            total_cost += neighbors[0][1] # Use the default cost if there are no parents
        else:
            min_cost = float("inf")
            for parents in parent_set:
                for neighbor in neighbors:
                    if neighbor[0] == list(parents):
                        cost = neighbor[1]
                        if cost < min_cost:
                            min_cost = cost
            total_cost += min_cost
    return total_cost

ordering = [5, 3, 1, 4, 2]
visited = {tuple(ordering): calculate_cost(ordering)}
queue = deque([ordering])
while queue:
    current_ordering = queue.popleft()
    current_cost = visited[tuple(current_ordering)]
    for i in range(len(current_ordering)):
        for j in range(i+1, len(current_ordering)):
            new_ordering = current_ordering[:i] + current_ordering[i:j+1][::-1] + current_ordering[j+1:]
            new_cost = calculate_cost(new_ordering)
            # Check if the new ordering is better than the current ordering
            if tuple(new_ordering) not in visited or new_cost < visited[tuple(new_ordering)]:
                visited[tuple(new_ordering)] = new_cost
                queue.append(new_ordering)

best_ordering = min(visited, key=visited.get)
best_cost = visited[best_ordering]

print("Best Ordering:", best_ordering)
print("Best Cost:", best_cost)


Best Ordering: (4, 5, 3, 1, 2)
Best Cost: 465.4339999999999


# Depth First Search

In [55]:
import math
from collections import deque

# Define the data
data = [(1, [([], 153.466), ([3], 96.093), ([4], 97.913), ([5], 99.835)]),
        (2, [([], 141.023), ([3], 122.446), ([4], 121.576), ([5], 123.398)]),
        (3, [([], 169.482), ([1], 112.109), ([2], 150.906), ([1, 2], 107.516), ([4], 51.681), ([5], 41.775)]),
        (4, [([], 169.482), ([1], 113.929), ([2], 150.036), ([1, 2], 108.982), ([3], 51.681), ([5], 36.188)]),
        (5, [([], 169.802), ([1], 116.171), ([2], 152.178), ([1, 2], 111.473), ([3], 42.096), ([4], 36.508)])]

def calculate_cost(ordering):
    total_cost = 0
    for vertex, neighbors in data:
        parent_set = set()
        for neighbor in reversed(neighbors):
            if set(tuple(neighbor[0])).issubset(set(ordering[:ordering.index(vertex)])):
                parent_set.add(tuple(neighbor[0]))
        if not parent_set:
            total_cost += neighbors[0][1] # Use the default cost if there are no parents
        else:
            min_cost = float("inf")
            for parents in parent_set:
                for neighbor in reversed(neighbors):
                    if neighbor[0] == list(parents):
                        cost = neighbor[1]
                        if cost < min_cost:
                            min_cost = cost
            total_cost += min_cost
    return total_cost

ordering = [5, 3, 1, 4, 2]
visited = {tuple(ordering): calculate_cost(ordering)}
stack = [ordering]
while stack:
    current_ordering = stack.pop()
    current_cost = visited[tuple(current_ordering)]
    for i in reversed(range(len(current_ordering))):
        for j in reversed(range(i+1, len(current_ordering))):
            new_ordering = current_ordering[:i] + current_ordering[i:j+1][::-1] + current_ordering[j+1:]
            new_cost = calculate_cost(new_ordering)
            # Check if the new ordering is better than the current ordering
            if tuple(new_ordering) not in visited or new_cost < visited[tuple(new_ordering)]:
                visited[tuple(new_ordering)] = new_cost
                stack.append(new_ordering)

best_ordering = min(visited, key=visited.get)
best_cost = visited[best_ordering]

print("Best Ordering:", best_ordering)
print("Best Cost:", best_cost)


Best Ordering: (4, 5, 3, 1, 2)
Best Cost: 465.4339999999999
